# Model proto

This is a rough prototype of our RL solution

In [ ]:
import os
if os.getcwd() == '/kaggle/working':
    import sys
    
    !rm -rf 'PMLDL-Project'
    !git clone -b main https://github.com/Sambura/PMLDL-Project
    
    if 'PMLDL-Project' not in sys.path: sys.path.append('PMLDL-Project')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np
import time
import sys
import os

if '..' not in sys.path: sys.path.append('..')

from src.utils.common import *
from src.utils.plot_utils import *
from src.dnd.actions import *
from src.dnd.units import *
from src.dnd.game_utils import *
from src.agent.agent_pg import DnDAgentPolicyGradient
from src.agent.deep_q_network import *
from src.agent.agent_utils import get_legal_action_resolver, get_states, agents_play_loop
from src.agent.training import *
from src.dnd.game_configs import *

In [ ]:
seed_everything(1234, deterministic_cudnn=True)

## Sample agent game

In [ ]:
def train_loop_sequential_V1_PG(agent: DnDAgent, 
                                game: DnDBoard,
                                reward_fn: callable,
                                iter_limit: int=10000,
                                raise_on_limit: bool=False) -> int:
    if not agent.sequential_actions:
        raise RuntimeWarning('Provided agent is incompatible with this train loop')

    # enemy_memory = []
    for iter_count in range(iter_limit):
        unit, player_id = game.current_unit, game.current_player_id

        while game.current_movement_left > 0 or not game.used_action: # while unit is still able to do something
            state, action_vector, new_coords, action = get_states_seq(game, agent)
            action_legal, move_legal = None, None

            if new_coords is not None: # move to new_coords
                move_legal, updates = game.move(new_coords, raise_on_illegal=False)
                finish_turn = not move_legal
            elif action is not None: # invoke the action
                action_legal, updates = game.use_action(action, raise_on_illegal=False)
                finish_turn = not action_legal
            else:
                finish_turn = True
                updates = None

            game_state = game.get_game_state(player_id)
            reward = reward_fn(game, game_state, unit, player_id, move_legal, action_legal, updates)
            if player_id == 0: 
                agent.memorize(state, action_vector, reward)
            # else:
            #     enemy_memory.append((state, action_vector, reward))

            if game_state != GameState.PLAYING: 
                # agent.learn()

                # for memory in enemy_memory:
                #     agent.memorize(*memory)
                # agent.learn()
                
                return iter_count + 1

            if finish_turn: break

        game.finish_turn()
    
    if raise_on_limit: raise RuntimeError('Iteration limit exceeded')

    # agent.learn()
# 
    # for memory in enemy_memory:
    #     agent.memorize(*memory)
    # agent.learn()

    return iter_limit

In [ ]:
def calculate_reward_classic_seq_PG(game, game_state, unit: Unit, player_id: int, move_legal: bool, action_legal: bool, updates: dict):
    if move_legal is None and action_legal is None: return -0.1

    if len(game.players_to_units[player_id]) == 0:
        return -1
    
    if len(game.players_to_units[player_id]) == len(game.units):
        return 1
    
    return 0

In [ ]:
board_size, game_config = get_2v2_1_config()

In [ ]:
yi, xi = np.meshgrid(np.arange(board_size[0]), np.arange(board_size[1]), indexing='ij')

def dnd_legal_moves_filter(state, probs):
    remaining_speed = state[11, 0, 0]

    current_unit_pos = np.where(state[2] != 0)
    y, x = current_unit_pos[0][0], current_unit_pos[1][0]        
    distance = np.abs(yi - y) + np.abs(xi - x)

    attack_range = state[4, y, x]
    possible_targets = np.where(np.logical_and(state[1], distance <= attack_range))

    occupied = np.logical_or(state[0], state[1])
    possible_positions = np.where(np.logical_and(distance <= remaining_speed, occupied == 0))

    filtered_map = np.zeros_like(probs)

    filtered_map[0, possible_positions[0], possible_positions[1]] = probs[0, possible_positions[0], possible_positions[1]]
    filtered_map[1, possible_targets[0], possible_targets[1]] = probs[1, possible_targets[0], possible_targets[1]]
    filtered_map[2, 0, 0] = probs[2, 0, 0]

    return filtered_map

In [ ]:
#board_size = (8, 8)

agent = DnDAgentPolicyGradient(board_shape=board_size, in_channels=DnDBoard.STATE_CHANNEL_COUNT, out_actions=3, 
    lr=1e-4, 
    gamma=0.99,
    batch_size=320000, 
    memory_capacity=50000,
    model_class=DnDEvalModelRT19,
    sequential_actions=True,
    legal_moves_filter=dnd_legal_moves_filter
)

train_loop = train_loop_sequential_V1_PG
reward_policy = calculate_reward_classic_seq_PG

In [ ]:
games = 50000
checkpoint_interval = 1000
game_iter_limit = 2500
average_result_over = 1000

# set to None to disable report / figures export
output_path = '../rnd/'
generation_name = '_temp-pg'
gen_header = \
'''# New field generation test

'''

In [ ]:
iters, checkpoints, rewards = [], [], []
mag1s, mag2s = [], []

def pg_memorize(state, actions, reward, new_state, game_over):
    agent.memorize(state, actions, reward)

gen = fieldGenerator(board_size=board_size).load_from_folder('../Tokens')

report_results = output_path is not None
if report_results:
    gen_path = os.path.join(output_path, generation_name)
    os.makedirs(gen_path, exist_ok=generation_name.startswith('_temp'))
else: gen_path = '..' # so that checkpoints still work

In [ ]:
last_update = time.time()

last_checkpoint = checkpoints[-1] if len(checkpoints) > 0 else 0
with tqdm(range(games), desc='Training', smoothing=0.1) as pbar:
    for i in pbar:
        gen.reset()
        game = gen.generate_balanced_game(targetCR=1)
        
        # outputs = agent.predict(game.observe_board())
        # metric = np.mean(np.abs(outputs), axis=(1, 2))
        # mag1s.append(metric[0])
        # mag2s.append(metric[1])
        # pbar.set_postfix({'mag1': metric[0], 'mag2': metric[1]})

        iter_count = train_loop(agent, game, reward_policy, iter_limit=game_iter_limit, raise_on_limit=False)
        iters.append(iter_count)
        rewards.append(np.mean(agent.reward_memory[:agent.memory_position]))
        agent.learn()

        if i == 10000:
            agent.set_lr(1e-5)

        ctime = time.time()
        if ctime - last_update > 20:
            clear_output(wait=True)
            plot_training_history(iters, checkpoints=checkpoints, min_ymax=20, average_last=average_result_over)
            #plt.plot(rewards)
            plt.plot(np.convolve(rewards, np.ones(50) / 50, mode='same')[:-25])
            plt.axhline(0, c='k')
            plt.show()
            if hasattr(pbar, 'container'): display(pbar.container)
            last_update = ctime

        iter_i = len(iters)
        if iter_i - last_checkpoint >= checkpoint_interval:
            last_checkpoint = iter_i
            checkpoints.append(iter_i)
            avg = np.mean(iters[-average_result_over:]) # recent performance
            agent.save_agent(os.path.join(gen_path, f'checkpoints/agent-{avg:0.1f}i-{iter_i / 1000:0.1f}k'))

In [ ]:
plot_training_history(iters, eps, checkpoints, vlines=reps, min_ymax=20, figsize=(20, 10), show=False)
if report_results:
    plt.savefig(os.path.join(gen_path, 'training.png'), bbox_inches='tight')
plt.show()

In [ ]:
plt.plot(mag1s, label='coords')
plt.plot(mag2s, label='action')
# plt.yscale('log')
plt.legend()
if report_results:
    plt.savefig(os.path.join(gen_path, 'magnitude.png'), bbox_inches='tight')
plt.show()
print(f'Total number of iterations: {np.sum(iters)}')

In [ ]:
if report_results:
    actual_games = len(iters)

    with open(os.path.join(gen_path, 'report.md'), 'w') as report:
        avg = np.mean(iters[-1000:])
        total = np.sum(iters)
        perf = 1e9 / (avg * total)
        report.write(
f'''{gen_header}
# Results
![image](training.png "")

* Average iteration count after {actual_games / 1000:0.1f}k games: {avg:0.2f}
* Total iterations performed: {total}
* Relative training performance: {perf:0.2f}
> 1e9 / (total_iter * avg_iter)

## Model's outputs magnitude history:

![magnitudes](magnitude.png "")

# Configuration

```python


games = {actual_games}
```
''')

In [ ]:
print(f'Shortest game: {np.min(iters)}')
print(f'Longest game: {np.max(iters)}')
print(f'Number of shortest games: {np.sum(np.array(iters) == np.min(iters))}')

In [ ]:
gen.reset()
game, colormap = decorate_game(generate_balanced_game(board_size, game_config))

agents_play_loop([agent, agent], game, colormap, reset_epsilon=False, delay=1)

# Visualization

In [ ]:
gen.reset()
game, colormap = decorate_game(generate_balanced_game(board_size, game_config))
print_game(game, colormap)

In [ ]:
state = game.observe_board()
plot_featuremaps(
    state, 
    title='Observed state', 
    fm_names=game.CHANNEL_NAMES,
    vmin = -3,
    vmax = 3,
    separate_cbars=True,
    cmap='Greens'
)

In [ ]:
state = game.observe_board()
output = agent.predict_probabilities(state)

plot_featuremaps(
    output, 
    title='Agent output', 
    fm_names=['New coords', 'Target unit', 'pass'],
    separate_cbars=True
)

print(agent.choose_single_action(state))

In [ ]:
seed_everything(0)
game, colormap = decorate_game(generate_balanced_game(board_size, game_config))
print_game(game, colormap)

In [ ]:
from src.agent.agent_utils import agent_take_turn
agent_take_turn(game, agent, None, True)
#print_game(game, colormap)

In [ ]:
plot_featuremaps(agent.predict(game.observe_board()))